# Strategy Construction Tool
This is to produce trade reports that can be used in the evaluation section to backtest.

In [1]:
!pip -q install --force-reinstall langgraph langchain_aws langchain-huggingface langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bbq-tools 1.39.0 requires python<4.0,>=3.9, which is not installed.
bqapi 0.36.0 requires python>=3.9, which is not installed.
bql-enterprise 0.17.0a0 requires pybql, which is not installed.
pygwalker 0.4.7 requires gw-dsl-parser==0.1.45a6, which is not installed.
flask-appbuilder 4.5.3 requires SQLAlchemy<1.5, but you have sqlalchemy 2.0.40 which is incompatible.
aiobotocore 2.13.3 requires botocore<1.34.163,>=1.34.70, but you have botocore 1.38.3 which is incompatible.
apache-airflow 2.9.2 requires sqlalchemy<2.0,>=1.4.36, but you have sqlalchemy 2.0.40 which is incompatible.
awscli 2.15.37 requires cryptography<40.0.2,>=3.3.2, but you have cryptography 40.0.2 which is incompatible.
awscli 2.15.37 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.
blis 1.0.1 re

In [2]:
import copy
import pandas as pd
import company_data as cd
import financial_agent as fa
import committee_agent as ca
import importlib
from tqdm import tqdm
import json

from utils.s3_helper import S3Helper
from datetime import datetime

import concurrent.futures


In [3]:
importlib.reload(fa)
importlib.reload(ca)

<module 'committee_agent' from '/project/committee_agent.py'>

### Collect the data needed to run the Agents
The Agents will require financial statement datasets and news datasets in order to conduct their analysis.

In [4]:
# Get the company reports
security_datasets = cd.SecurityData('tmp/fs', 'dow_quarterly_ltm_v2.json')
# get the saved news datasets
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')
# Convert to pandas dataframe
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

### Run the Agentic Models
Running the inference tasks across 893 date/ security combinations in 1:32h.

In [5]:
financial_agent = fa.FinancialAnalystAgent()
committee_agent = ca.CommitteeAgent()

In [6]:
def run_single(security: str, as_of_date: str) -> dict:
    """
    Function to run a single run of the Agent
    """
    company_data = security_datasets.get_security_all_data(as_of_date, security)
    # Time the run
    start_time = datetime.now()
    # Run the financial analyst agent
    financial_report = financial_agent.run(security_data=company_data, 
                                       news_data=news_headlines, 
                                       as_of_date=as_of_date)
    # Run the committee agent
    committee_report = committee_agent.run(senior_analyst_report=financial_report['senior_report'],
                                           financial_statement_analysis=financial_report['financial_report'],
                                           security_data=company_data)
    end_time = datetime.now()
    decision_dict = {
        'date': as_of_date,
        'security': security,
        'earning_decision': financial_report['final_output'].direction,
        'earning_magnitude': financial_report['final_output'].magnitude,
        'earning_confidence': financial_report['final_output'].confidence,
        'recommendation': committee_report['results'].recommendation,
        
        'responses': {'financial_analyst': financial_report,
                     'committee_report': committee_report},
        'time': str(end_time - start_time)
    }
    return decision_dict
    
    

In [7]:
dates_and_securities = security_datasets.date_security_timeseries()
test_security = dates_and_securities[0]

In [7]:
test_security

{'date': '2020-04-24', 'security': 'AXP UN Equity'}

In [19]:
test_output = run_single('AXP UN Equity', '2020-04-24')

In [7]:
data_output = []
def backtest(dates_and_securities):
    # create the securities
    progress = tqdm(total=len(dates_and_securities), position=0, leave=True)
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(run_single, single['security'], single['date']) for single in dates_and_securities]
        for f in concurrent.futures.as_completed(futures):
            progress.update(n=1)
            data_output.append(f.result())
        

In [14]:
# Run the backtest
backtest(dates_and_securities[228:])

 13%|█▎        | 84/665 [14:44<1:50:51, 11.45s/it] 

KeyboardInterrupt: 

### Save the raw output to S3

In [20]:
data_analysis_final = copy.deepcopy(data_output)

In [21]:
# Some of the items are not serializable. Convert to string before JSON
for item in data_analysis_final:
    item['responses']['financial_analyst']['final_output'] = str(item['responses']['financial_analyst']['final_output'])
    item['responses']['committee_report']['results'] = str(item['responses']['committee_report']['results'])
    item['responses']['committee_report']['history'] = str(item['responses']['committee_report']['history'])
    item['time']=str(item['time'])

In [18]:
with open('/tmp/agentic_output_claude_2.json', 'w') as f:
    json.dump(data_analysis_final, f)

In [19]:
s3_helper.add_file(local_filename='/tmp/agentic_output_claude_2.json')

### Create the Trade Report

In [23]:
def trade_report_generation(data, trade_only=False) -> pd.DataFrame:
    """
    Function to create the trade report to pass into the Strategy Analysis tool
    """
    trade_report = []
    if trade_only:
        for item in data:
            trade_report.append({'date': item['date'], 
                                 'security': item['security'], 
                                 'decision': item['recommendation'],
                                 'confidence': item['earning_confidence']})
    
        return pd.DataFrame(data=trade_report)
    else:
        for item in data:
            trade_report.append({'date': item['date'], 
                                 'security': item['security'], 
                                 'decision': item['recommendation'],
                                 'confidence': item['earning_confidence'],
                                 'earning_decision': item['earning_decision'],
                                 'earning_magnitude': item['earning_magnitude']})
        return trade_report

In [24]:
trades = trade_report_generation(data_analysis_final)

In [25]:
with open('Results/Agentic/trades_claude.json', 'w') as f:
    json.dump(trades, f)

In [22]:
len(data_output)

311